<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example:-Examine-Potentials" data-toc-modified-id="Example:-Examine-Potentials-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Example: Examine Potentials</a></span></li><li><span><a href="#Example:-Manually-Examining-Simulation-Information" data-toc-modified-id="Example:-Manually-Examining-Simulation-Information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example: Manually Examining Simulation Information</a></span></li><li><span><a href="#Example:-Automatically-Creating-a-Simulation-Script" data-toc-modified-id="Example:-Automatically-Creating-a-Simulation-Script-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example: Automatically Creating a Simulation Script</a></span></li><li><span><a href="#Example:-Analysis" data-toc-modified-id="Example:-Analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Example: Analysis</a></span></li></ul></div>

In [ ]:
import numpy as np
import utils
import matplotlib.pyplot as plt
%matplotlib inline

# Example: Examine Potentials

In [ ]:
from utils import LAMMPS

In [ ]:
r_, u_, f_ = LAMMPS.force_shifted_lennard_jones(
    r_min=0.7, r_cut=2.5, eps=1.0, sigma=1.0, alpha=6, bins=1000)

In [ ]:
plt.plot(r_, u_, color='C0')
plt.plot(r_, f_, color='C1')
plt.ylim(-3.5, 3.5)

In [ ]:
r_, u_, f_ = LAMMPS.force_shifted_yukawa(
    r_min=0, r_cut=2.5, eps=1.0, kappa=5.0, bins=1000)

In [ ]:
plt.plot(r_, u_, color='C0')
plt.plot(r_, f_, color='C1')

In [ ]:
sig = 1.0
kappa_sig = 2.5
r_, u_, f_ = LAMMPS.force_shifted_tanh(
    r_min=0, r_cut=2.5*sig, eps=-1.0, sigma=sig, kappa=kappa_sig/sig, bins=1000)

In [ ]:
plt.plot(r_, u_, color='C0')
plt.plot(r_, f_, color='C1')

# Example: Manually Examining Simulation Information

In [ ]:
# Step 1: Load Colloid from C++ code
from utils import Colloid

c = Colloid()
c.load(
    filename='../examples/initialize_colloid/colloid.json'
)

# Scale so motif points are "at contact" - also sets "diameters"
# of the beads when using inplace=True
scale_constant = c.scale(inplace=True) 

# Always save the Colloid object so you have the mapping of types
c.save('colloid.pkl')

In [ ]:
# Step 2a: Create a mixture of a set of different enantiomorphs
from utils import LAMMPS

buffer = 1.0 # Gap around colloids
box = [175, 175] # Box widths
n_each = [100, 100] # Number of each chirality [same, opposite]

coords_, types_, bbox_ = LAMMPS.tile(c, box, buffer, n=n_each)

In [ ]:
# Step 2b: Let's color by chirality
from utils import Analysis

%matplotlib notebook

l = len(c.coords)
end = l
while end <= len(coords_):
    x = Analysis.identify_chirality(
        Analysis.unwrap(c.coords, box), 
        Analysis.unwrap(coords_[end-l:end], box)
    )

    plt.plot(coords_[end-l:end,0], coords_[end-l:end,1], '.', color='C{}'.format(int(not x)))
    end += l

# Example: Automatically Creating a Simulation Script

In [ ]:
# Step 3: Turn that into a LAMMPS file to read
init_config_filename = 'init_config.lammps'
_ = LAMMPS.create_initial_configuration(
    c, 
    box, 
    buffer, 
    n_each, 
    init_config_filename
)

In [ ]:
# Step 4: Record the potential for each pair of particles
bins = 1000
potential_filename = 'potentials.table'
max_rcut, rcut_dict = LAMMPS.tabulate_potentials(
    c,
    filename=potential_filename, 
    alpha=6, 
    #kappa_sigma=2.5,
    bins=bins,
    style='fslj'
)

pseudo_diameter = LAMMPS.rigid_body_min_comm(c, box, eps=0.1) + max_rcut

In [ ]:
import re
def make(
    colloid, 
    output_filename,
    template_filename, 
    init_config_filename,
    potential_filename,
    bins,
    pseudo_diameter,
    rcut_dict,
    steps_per_loop=10000,
    n_init_loops=50,
    n_equil_loops=500,
    n_prod_loops=50000,
    rec_freq=500000,
    res_freq=500000,
    Tstart=1.0,
    Tfinal=0.3,
    Pstart=0.01,
    Pfinal=0.01,
    dt=0.005,
):
    raw = open(template_filename, 'r').read()
    data = re.sub('__INIT_CONFIG__', str(init_config_filename), raw)
    data = re.sub('__REC_FREQ__', str(rec_freq), data)
    data = re.sub('__RES_FREQ__', str(res_freq), data)
    data = re.sub('__TSTART__', str(Tstart), data)
    data = re.sub('__TFINAL__', str(Tfinal), data)
    data = re.sub('__TDAMP__', str(100*dt), data) l
    data = re.sub('__PSTART__', str(Pstart), data)
    data = re.sub('__PFINAL__', str(Pfinal), data)
    data = re.sub('__PDAMP__', str(100000*dt), data)
    data = re.sub('__TIMESTEP__', str(dt), data)
    data = re.sub('__TABLE_BINS__', str(int(bins)), data)
    data = re.sub('__PSEUDODIAMETER__', str(pseudo_diameter), data)
    data = re.sub('__STEPS_PER_LOOP__', str(int(steps_per_loop)), data)
    data = re.sub('__N_INIT_LOOPS__', str(int(n_init_loops)), data)
    data = re.sub('__N_EQUIL_LOOPS__', str(int(n_equil_loops)), data)
    data = re.sub('__N_PROD_LOOPS__', str(int(n_prod_loops)), data)

    n_atom_types = len(colloid.forward)
    
    command = ""
    for i in range(1, n_atom_types + 1):
        for j in range(i, n_atom_types + 1):
            command += "pair_coeff\t{} {} {} {}_{}\n".format(
                i, 
                j, 
                potential_filename, 
                i, 
                j
            )
    data = re.sub('__PAIR_COEFF__', command, data)
    
    command = ""
    for i in range(1, n_atom_types + 1):
        for j in range(i, n_atom_types + 1):
            command += "pair_write\t{} {} {} rsq {} {} potentials.interpolated {}_{}\n".format(
                i, 
                j, 
                int(bins), 
                rcut_dict['{}_{}'.format(i,j)][0], 
                rcut_dict['{}_{}'.format(i,j)][1],
                i,
                j
            )
    data = re.sub('__PAIR_WRITE__', command, data)

    open(output_filename, 'w').write(data)

In [ ]:
make(
    colloid=c,
    output_filename='input.lmp',
    template_filename='npt.template', 
    potential_filename=potential_filename,
    init_config_filename=init_config_filename,
    bins=bins,
    pseudo_diameter=pseudo_diameter,
    rcut_dict=rcut_dict
)

# Example: Analysis

In [ ]:
for i in range(14, 15):
    for j in range(i, 14+1):
        plt.figure()
        r_u_f_a = LAMMPS.read_lammps_potential(
            'potentials.table', i, j)
        r_u_f_b = LAMMPS.read_lammps_potential(
            'potentials.interpolated', i, j)
        
        plt.plot(r_u_f_a[:,0], r_u_f_a[:,1], color='C0', alpha=0.4)
        plt.plot(r_u_f_a[:,0], r_u_f_a[:,2], color='C1', alpha=0.4)
        
        plt.plot(r_u_f_b[:,0], r_u_f_b[:,1], color='C3', alpha=0.4)
        plt.plot(r_u_f_b[:,0], r_u_f_b[:,2], color='C4', alpha=0.4)
        plt.ylim(-2.5, 2.5)

In [ ]:
# from utils import Analysis
# Analysis.sanity_checks('log.thermo') # Sanity checks

In [ ]:
# steps_per_second = Analysis.performance('log.thermo')
# print('{} +/- {}'.format(np.mean(steps_per_second), np.std(steps_per_second)))

In [ ]:
results = Analysis.thermo('log.thermo')

In [ ]:
%matplotlib inline

fig, ax1 = plt.subplots()

ax1.plot(results['Step'], results['Temp'], color='C0')
ax1.set_ylabel('Temperature', color='C0')
ax2 = ax1.twinx()  
ax2.plot(results['Step'], results['PotEng'], color='C1')
ax2.set_ylabel('PotEng', color='C1')

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(results['Step'], results['Volume'], color='C0')
ax1.set_ylabel('Volume', color='C0')
ax2 = ax1.twinx()  
ax2.plot(results['Step'], results['Press'], color='C1')
ax2.set_ylabel('Press', color='C1')

In [ ]:
plt.plot(results['Temp'], results['PotEng'], '.')

delta = (np.max(results['PotEng']) - np.min(results['PotEng']))
x = delta/2. + np.min(results['PotEng'])
tol = 0.1*delta
plt.axhline(x, color='k')
plt.axhline(x+tol, color='gray')
plt.axhline(x-tol, color='gray')